## Week 10 and 11 Assignment - DATASCI200 Introduction to Data Science Programming, UC Berkeley MIDS

Write code in this Jupyter Notebook to solve the following problems. Please upload this **Notebook** with your solutions to your GitHub repository and provide a link in the last question in gradescope. 

Assignment due date: 11:59PM PT the night before the Week 12 Live Session. Do **NOT** push/upload the data file. 

## Objectives

- Explore and glean insights from a real dataset using pandas
- Practice using pandas for exploratory analysis, information gathering, and discovery
- Practice cleaning data and answering questions

## General Guidelines:

- This is a **real** dataset and so it may contain errors and other pecularities to work through
- This dataset is ~218mb, which will take some time to load (and probably won't load in Google Sheets or Excel)
- If you make assumptions, annotate them in your responses
- While there is one code/markdown cell positioned after each question as a placeholder, some of your code/responses may require multiple cells
- Double-click the markdown cells that say for example **1a answer here:** to enter your written answers. If you need more cells for your written answers, make them markdown cells (rather than code cells)
- This homework assignment is not autograded because of the variety of responses one could give. 
  - Please upload this notebook to the autograder page and the TAs will manually grade it. 
  - Ensure that each cell is run and outputs your answer for ease of grading! 
  - Highly suggest to do a `restart & run all` before uploading your code to ensure everything runs and outputs correctly.
  - Answers without code (or code that runs) will be given 0 points.
- **This is meant to simulate real world data so you will have to do some external research to determine what some of the answers are!** 

## Dataset

You are to analyze campaign contributions to the 2016 U.S. presidential primary races made in California. Use the csv file located here: https://drive.google.com/file/d/1Lgg-PwXQ6TQLDowd6XyBxZw5g1NGWPjB/view?usp=sharing. You should download and save this file in the same folder as this notebook is stored.  This file originally came from the U.S. Federal Election Commission (https://www.fec.gov/).

**DO NOT PUSH THIS FILE TO YOUR GITHUB REPO!**

- Best practice is to not have DATA files in your code repo. As shown below, the default load is outside of the folder this notebook is in. If you change the folder where the file is stored please update the first cell!
- If you do accidentally push the file to your github repo - follow the directions here to fix it: https://docs.google.com/document/d/15Irgb5V5G7pKPWgAerH7FPMpKeQRunbNflaW-hR2hTA/edit?usp=sharing

Documentation for this data can be found here: https://drive.google.com/file/d/11o_SByceenv0NgNMstM-dxC1jL7I9fHL/view?usp=sharing

## Data Questions

You are working for a California state-wide election campaign. Your boss wants you to examine historic 2016 election contribution data to see what zipcodes are more supportive of fundraising for your candidate. 

Your boss asks you to filter out some of the records:
- Only use primary 2016 contribution data (more like how your race is).
- Concentrate on Bernie Sanders as a candidate (most a like your candidate)

The questions your boss wants answered is:
- Which zipcode (5-digit zipcode) had the highest count of contributions and the most dollar amount?
- What day(s) of the month do most people donate?

## Setup

Run the cell below as it will load the data into a pandas dataframe named `contrib`. Note that a custom date parser is defined to speed up loading. If Python were to guess the date format, it would take even longer to load.

In [123]:
import pandas as pd
import numpy as np
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_rows', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# ✅ Define 19 columns to match your file (includes trailing comma)
columns = [
    'cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city', 'contbr_st',
    'contbr_zip', 'contbr_employer', 'contbr_occupation', 'contb_receipt_amt',
    'contb_receipt_dt', 'receipt_desc', 'memo_cd', 'memo_text',
    'form_tp', 'file_num', 'tran_id', 'election_tp', 'extra_col'  # extra_col absorbs the extra comma
]

# ✅ Load the CSV with corrected column count
contrib = pd.read_csv('P00000001-CA.csv', names=columns, header=0, engine='python')

# Clean string columns
contrib = contrib.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Convert date and amount
contrib['contb_receipt_amt'] = pd.to_numeric(contrib['contb_receipt_amt'], errors='coerce')
contrib['contb_receipt_dt'] = pd.to_datetime(contrib['contb_receipt_dt'], errors='coerce')

# Check election_tp values
print("Unique values in election_tp:", contrib['election_tp'].unique())

# Filter for Bernie Sanders and Primary 2016
bernie = contrib[
    (contrib['cand_nm'] == 'Sanders, Bernard') &
    (contrib['election_tp'] == 'P2016')
].copy()
print("Bernie primary records:", bernie.shape)

# Clean and filter ZIP codes (California range)
bernie['zipcode_5'] = bernie['contbr_zip'].astype(str).str.extract(r'(\d{5})')
bernie['zipcode_5'] = pd.to_numeric(bernie['zipcode_5'], errors='coerce')
bernie = bernie[(bernie['zipcode_5'] >= 90001) & (bernie['zipcode_5'] <= 96162)]

# Remove negative donations
bernie = bernie[bernie['contb_receipt_amt'] >= 0]

# Extract day of month
bernie['donation_day'] = bernie['contb_receipt_dt'].dt.day

# Top ZIP by contributions
zip_summary = bernie.groupby('zipcode_5').agg(
    total_contributions=('contb_receipt_amt', 'sum'),
    contribution_count=('contb_receipt_amt', 'count')
).sort_values(by='total_contributions', ascending=False)

print("\n ZIP code with highest total contributions:")
print(zip_summary.head(1))

# Most common donation days
donation_day_counts = bernie['donation_day'].value_counts().sort_values(ascending=False)

print("\n  Most common donation days of the month:")
print(donation_day_counts.head(5))


/var/folders/6q/dd_6hhkd2nd4q1pyp1nh1d140000gn/T/ipykernel_2703/4035942265.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  contrib['contb_receipt_dt'] = pd.to_datetime(contrib['contb_receipt_dt'], errors='coerce')


Unique values in election_tp: ['P2016' 'G2016' nan 'P2020']
Bernie primary records: (407171, 19)

 ZIP code with highest total contributions:
           total_contributions  contribution_count
zipcode_5                                         
94110               294,061.13                4046

  Most common donation days of the month:
donation_day
29    23508
30    22899
31    19486
14    17793
8     16774
Name: count, dtype: int64


***
## 1. Initial Data Checks (50 points)

First we will take a preliminary look at the data to check that it was loaded correctly and contains the info we need.

The questions to answer at the end of this section:
- Do we have the correct # of columns and rows. 
- Do the records contain data for the questions we want to answer 
- What columns are important? 
- What columns can be dropped?
- What are the data problems?

**1a.** Print the *shape* of the data. Does this match the expectation? (2 points)

In [104]:
# 1a YOUR CODE HERE
print(contrib.shape)

(1125659, 19)


- **1a answer here:** 

**1b.** Print a list of column names. Are all the columns included that are in the documentation? (2 points)

In [105]:
# 1b YOUR CODE HERE
print(contrib.columns.tolist())


['cmte_id', 'cand_id', 'cand_nm', 'contbr_nm', 'contbr_city', 'contbr_st', 'contbr_zip', 'contbr_employer', 'contbr_occupation', 'contb_receipt_amt', 'contb_receipt_dt', 'receipt_desc', 'memo_cd', 'memo_text', 'form_tp', 'file_num', 'tran_id', 'election_tp', 'extra_col']


- **1b answer here:** 

**1c** Print out the first five rows of the dataset. How do the columns `cand_id`, `cand_nm` and `contbr_st` look? (3 points)

In [106]:
# 1c YOUR CODE HERE
print(contrib[['cand_id', 'cand_nm', 'contbr_st']].head())

     cand_id                  cand_nm contbr_st
0  P00003392  Clinton, Hillary Rodham        CA
1  P00003392  Clinton, Hillary Rodham        CA
2  P00003392  Clinton, Hillary Rodham        CA
3  P60007168         Sanders, Bernard        CA
4  P60007168         Sanders, Bernard        CA


- **1c answer here:** 

**1d.** Print out the values for the column `election_tp`. In your own words, based on the documentation, what information does the `election_tp` variable contain? Do the values in the column match the documentation? (3 points)

In [107]:
# 1d YOUR CODE HERE
print(contrib['election_tp'])

0          P2016
1          P2016
2          P2016
3          P2016
4          P2016
           ...  
1125654    P2016
1125655    P2016
1125656    P2016
1125657    P2016
1125658    P2016
Name: election_tp, Length: 1125659, dtype: object


- **1d answer here:** 

**1e.** Print out the datatypes for all of the columns. What are the datatypes for the `contbr_zip`, `contb_receipt_amt`, `contb_receipt_dt`? (5 points)

In [108]:
# 1e YOUR CODE HERE
print(contrib.dtypes)

cmte_id                      object
cand_id                      object
cand_nm                      object
contbr_nm                    object
contbr_city                  object
contbr_st                    object
contbr_zip                   object
contbr_employer              object
contbr_occupation            object
contb_receipt_amt           float64
contb_receipt_dt     datetime64[ns]
receipt_desc                 object
memo_cd                      object
memo_text                    object
form_tp                      object
file_num                      int64
tran_id                      object
election_tp                  object
extra_col                   float64
dtype: object


- **1e answer here:** 

**1f.** What columns have the most non-nulls?  Would you recommend to drop any columns based on the number of nulls? (5 points)

In [109]:
# 1f YOUR CODE HERE
non_null_counts = contrib.notnull().sum().sort_values(ascending=False)
print(non_null_counts)


cmte_id              1125659
cand_id              1125659
cand_nm              1125659
contbr_nm            1125659
contbr_st            1125659
contb_receipt_amt    1125659
contb_receipt_dt     1125659
file_num             1125659
tran_id              1125659
form_tp              1125659
contbr_city          1125633
contbr_zip           1125564
election_tp          1124234
contbr_occupation    1115260
contbr_employer       967757
memo_text             501148
memo_cd               144268
receipt_desc           15045
extra_col                  0
dtype: int64


- **1f answer here:** 

**1g.** A column we know that we want to use is the cand_nm column.  From the documentation each candidate is a unique candidate id also. Check data quality of `cand_id` column to see if it matches `cand_nm` column. Specifically check to ensure our targetted candidate 'Bernard Sanders' always has the same cand_id throughout. Any issues with `cand_nm` matching `cand_id`? (5 points)

In [110]:
# 1g YOUR CODE HERE
# Get unique cand_id(s) for the name "Sanders, Bernard"
sand_id_check = contrib[contrib['cand_nm'] == 'Sanders, Bernard']['cand_id'].unique()
print("Candidate IDs for 'Sanders, Bernard':", sand_id_check)

# Group by candidate ID and get unique names per ID
cand_id_mapping = contrib.groupby('cand_id')['cand_nm'].nunique()
print("Number of unique names per cand_id:")
print(cand_id_mapping.sort_values(ascending=False))

# Show all candidate IDs with more than one name (just in case)
mismatches = contrib.groupby('cand_id')['cand_nm'].unique()
mismatches = mismatches[mismatches.apply(len) > 1]
print("Any cand_id associated with multiple names:")
print(mismatches)


Candidate IDs for 'Sanders, Bernard': ['P60007168']
Number of unique names per cand_id:
cand_id
P00003392    1
P20002671    1
P20002721    1
P20003281    1
P20003984    1
P40003576    1
P60003670    1
P60005915    1
P60006046    1
P60006111    1
P60006723    1
P60007168    1
P60007242    1
P60007572    1
P60007671    1
P60007697    1
P60008059    1
P60008398    1
P60008521    1
P60008885    1
P60009685    1
P60022654    1
P80001571    1
P80003379    1
P80003478    1
Name: cand_nm, dtype: int64
Any cand_id associated with multiple names:
Series([], Name: cand_nm, dtype: object)


- **1g answer here:** 

**1h.** Another area to check is to make sure all of the records are from California. Check the `contbr_st` column - are there any records outside of California based on `contbr_st`? (5 points)

In [111]:
# 1h YOUR CODE HERE
# Show all unique states in the contributor state column
unique_states = contrib['contbr_st'].unique()
print("Unique values in contbr_st column:")
print(unique_states)



Unique values in contbr_st column:
['CA']


- **1h answer here:** 

**1i.** The next column to check for the analysis is the `tran_id` column. This column could be the primary key so look for duplicates. How many duplicate entries are there? Any pattern for why are there duplicate entries? (5 points)

In [112]:
# 1i YOUR CODE HERE
# Check for duplicate transaction IDs
duplicate_tran_ids = contrib[contrib['tran_id'].duplicated(keep=False)]

# Number of duplicate rows
print(f"Number of duplicate tran_id rows: {len(duplicate_tran_ids)}")


Number of duplicate tran_id rows: 6873


- **1i answer here:** 

**1j.** Another column to check is the `contb_receipt_amt` that shows the donation amounts. How many negative donations are included? What do negative donations mean? Please show at least pull a few rows to look at the records with negative donations. Do these records match with the expectation of why a negative donation would happen? (5 points)

In [113]:
# 1j YOUR CODE HERE
# Filter for negative donation amounts
negative_donations = contrib[contrib['contb_receipt_amt'] < 0]

# Count them
print(f"Number of negative donation records: {len(negative_donations)}")


Number of negative donation records: 11896


- **1j answer here:**

**1k.** One more column to look at is the date of donation column. Are there any dates outside of the primary period (defined as 1 Jan 2014 to 7 June 2016)? Are the dates well-formatted for our analysis? (5 points)

In [114]:
# 1k YOUR CODE HERE
from datetime import datetime

start_primary = datetime(2014, 1, 1)
end_primary = datetime(2016, 6, 7)

outside_primary = contrib[
    (contrib['contb_receipt_dt'] < start_primary) |
    (contrib['contb_receipt_dt'] > end_primary)
]

print(f" Number of records outside the primary window: {len(outside_primary)}")
print(outside_primary[['cand_nm', 'contbr_nm', 'contb_receipt_amt', 'contb_receipt_dt']].head())

missing_dates = contrib[contrib['contb_receipt_dt'].isnull()]
print(f"Number of records with missing or invalid dates: {len(missing_dates)}")





 Number of records outside the primary window: 451372
                         cand_nm          contbr_nm  contb_receipt_amt  \
9932                Rubio, Marco  WHEELER, MARY MS.             -20.00   
9994                Rubio, Marco  WHEELER, MARY MS.              20.00   
14673  Cruz, Rafael Edward 'Ted'    RANDALL, DICK J          -5,400.00   
14682              Johnson, Gary          BOAL, ROB              30.00   
14697              Johnson, Gary        LEE, JASCHA              88.45   

      contb_receipt_dt  
9932        2013-11-05  
9994        2013-11-05  
14673       2016-06-30  
14682       2016-07-20  
14697       2016-07-14  
Number of records with missing or invalid dates: 0


- **1k answer here:**

**1l.** Finally, answer the initial questions in the cells below (5 points)

Do we have the correct # of columns and rows.**1l.1** 

- **1l.1 answer here:**
Yes

**1l.2** Do the records contain data for the questions we want to answer?

- **1l.2 answer here:**
Yes


**1l.3** What columns are important?

- **1l.3 answer here:** 

cand_nm – Candidate name

cand_id – Candidate ID (for validation)

contbr_zip – Contributor ZIP code (for geographic analysis)

contb_receipt_amt – Donation amount

contb_receipt_dt – Date of donation

tran_id – Transaction ID (for duplicate detection)

contbr_st – Contributor state (to ensure CA-only data)

election_tp – Election type (to filter for P2016)

memo_cd – Memo code (helps interpret special transactions like refunds)

memo_text – Memo text (reason for refunds/redesignations)

**1l.4** What columns can be dropped?

- **1l.4 answer here:** 
contbr_city – You’re already using contbr_zip for location; city adds little extra value

contbr_employer – Not needed unless analyzing donor professions/industries

contbr_occupation – Same as above; optional demographic insight

receipt_desc – Often blank or contains minimal data

file_num – Administrative; not needed for analysis

form_tp – Internal form tracking; not relevant here

cmte_id – Committee info is not needed unless tracking PACs

memo_cd – Only keep if you’re not already using memo_text for interpreting negative donations

**1l.5** What are the data problems?

- **1l.5 answer here:**
 Data Problems Found
1. Trailing Commas Causing Column Misalignment
2. Missing or Invalid Dates
3. Negative Donations
4. Duplicate tran_id Values
5. Non-California Records
6. Multiple cand_id per cand_nm or vice versa
7. Missing or Nearly Empty Columns

**1l.6** List any assumptions so far:

- **1l.6 answer here:**
Each candidate has one unique cand_id

Dataset contains only California records (contbr_st == 'CA')

Only contributions between Jan 1, 2014 and June 7, 2016 are relevant

Negative contb_receipt_amt values are valid (e.g., refunds)

tran_id values are expected to be unique

Valid election type is P2016 only

ZIP code analysis uses the first 5 digits of contbr_zip

Data follows the 18-column FEC schema

memo_cd and memo_text are sparsely populated but useful



***
## 2. Data filtering and data quality fixes (30 points)

Now that we have a basic understanding of the data, let's filter out the records we don't need and fix the data.

**2a.** From the dataset filter out (remove) any election_tp not in the primary election. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [115]:
# 2a YOUR CODE HERE
# Filter the dataset to keep only primary election contributions
primary_only = contrib[contrib['election_tp'] == 'P2016'].copy()

# Show the new shape
print("Shape after filtering for primary election (P2016):", primary_only.shape)



Shape after filtering for primary election (P2016): (810481, 19)


**2b.** From the dataset filter out (remove) any candidate that is not Bernie Sanders. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [116]:
# 2b YOUR CODE HERE
# Further filter to include only Bernie Sanders' records
bernie_only = primary_only[primary_only['cand_nm'] == 'Sanders, Bernard'].copy()

# Show the new shape
print("Shape after filtering for Bernie Sanders:", bernie_only.shape)


Shape after filtering for Bernie Sanders: (407171, 19)


**2c.** The `contbr_zip` column is not formatted well for our analysis. Make a new zipcode column that is the five-digit zipcodes. Filter out any records outside of California based on the zipcode. Print/show the shape of the dataframe after the filtering is complete. (10 points).

- You will have to research what the valid 5-digit zipcodes for California are!

In [ ]:
# 2c YOUR CODE HERE
# Ensure 'contbr_zip' is treated as a string
bernie_only['contbr_zip'] = bernie_only['contbr_zip'].astype(str)

# Extract the first five digits
bernie_only['zipcode_5'] = bernie_only['contbr_zip'].str[:5]

# Display unique values to verify
print("Unique ZIP codes after extraction:", bernie_only['zipcode_5'].unique()[:5])

# Convert 'zipcode_5' to numeric, setting errors='coerce' will replace non-numeric values with NaN
bernie_only['zipcode_5'] = pd.to_numeric(bernie_only['zipcode_5'], errors='coerce')

# Define the range for California ZIP codes
ca_zip_min = 90001
ca_zip_max = 96162

# Filter for ZIP codes within the California range
bernie_ca = bernie_only[(bernie_only['zipcode_5'] >= ca_zip_min) & (bernie_only['zipcode_5'] <= ca_zip_max)].copy()

# Display the shape after filtering
print("Shape after filtering for California ZIP codes:", bernie_ca.shape)


# Check for any remaining non-California ZIP codes
non_ca_zips = bernie_ca[~((bernie_ca['zipcode_5'] >= ca_zip_min) & (bernie_ca['zipcode_5'] <= ca_zip_max))]

# Display if any non-CA ZIP codes remain
print("Non-California ZIP codes remaining:", non_ca_zips['zipcode_5'].unique())


Unique ZIP codes after extraction: ['93011' '90278' '92084' '92683' '94952']
Shape after filtering for California ZIP codes: (407135, 20)
Non-California ZIP codes remaining: []


**2d.** The receipt amount column has negative donations. After talking with your team, a decision was made that the best course of action is to remove these negative values so that the donation count and amount is more accurate. Print/show the shape of the dataframe after the filtering is complete. (5 points)

In [118]:
# 2d YOUR CODE HERE
# Filter out negative donation amounts
bernie_ca_clean = bernie_ca[bernie_ca['contb_receipt_amt'] >= 0].copy()

# Print the new shape of the dataset
print("Shape after removing negative donations:", bernie_ca_clean.shape)



Shape after removing negative donations: (404048, 20)


**2e.** From the dataset drop any columns that won't be used in the analysis. Print/show the shape of the dataframe after the dropping is complete. What columns did you drop and why? (5 points)

In [119]:
# 2e YOUR CODE HERE
# Drop unnecessary columns
bernie_final = bernie_ca_clean.drop(columns=[
    'cmte_id',          # Internal committee ID (not used in analysis)
    'cand_id',          # cand_nm is enough for this analysis
    'contbr_nm',        # Individual names not needed for ZIP-level summary
    'contbr_city',      # Redundant with ZIP
    'contbr_st',        # Already filtered to CA
    'contbr_zip',       # Replaced by zipcode_5
    'contbr_employer',  # Not needed unless analyzing by occupation
    'contbr_occupation',# Same as above
    'receipt_desc',     # Often blank
    'memo_cd',          # Used for internal tracking
    'memo_text',        # We removed negative values, so this is no longer useful
    'form_tp',          # Internal FEC form type
    'file_num',         # FEC-specific metadata
    'tran_id',          # Useful for deduping, but not needed for summary analysis
])

# Show shape after dropping columns
print("Shape after dropping unused columns:", bernie_final.shape)



Shape after dropping unused columns: (404048, 6)


- **2e answer here:**

    'cmte_id',          # Internal committee ID (not used in analysis)
    
    'cand_id',          # cand_nm is enough for this analysis

    'contbr_nm',        # Individual names not needed for ZIP-level summary

    'contbr_city',      # Redundant with ZIP

    'contbr_st',        # Already filtered to CA

    'contbr_zip',       # Replaced by zipcode_5

    'contbr_employer',  # Not needed unless analyzing by occupation

    'contbr_occupation',# Same as above

    'receipt_desc',     # Often blank

    'memo_cd',          # Used for internal tracking

    'memo_text',        # We removed negative values, so this is no longer useful

    'form_tp',          # Internal FEC form type

    'file_num',         # FEC-specific metadata
    
    'tran_id',          # Useful for deduping, but not needed for summary analysis

**2f.** List any assumptions that you made up to this point:

- **2f answer here:**

Only contributions with election_tp == 'P2016' are relevant to the 2016 primary election.

Only records where cand_nm == 'Sanders, Bernard' are relevant to your candidate analysis.

California ZIP codes fall within the range 90001–96162, and any records outside this range are not from California.

Only the first 5 digits of contbr_zip are used for analysis.

Negative donation amounts (e.g., refunds or redesignations) are not relevant for this analysis and should be removed.

The column contb_receipt_dt is correctly parsed as datetime64[ns] and used for time-based analysis.

Columns like memo_cd, memo_text, receipt_desc, etc. are not needed after negative values are removed.

Removing columns like tran_id, file_num, and form_tp does not impact the accuracy of ZIP-level or date-level aggregation.

Dropping non-essential columns simplifies analysis without removing necessary information.

***
## 3. Answering the questions (20 points)

Now that the data is cleaned and filterd - let's answer the two questions from your boss!

**3a.** Which zipcode had the highest count of contributions and the most dollar amount? (10 points)

In [121]:
# 3a YOUR CODE HERE
# Group by 5-digit ZIP and calculate total contributions and count
zip_summary = bernie.groupby('zipcode_5').agg(
    total_contributions=('contb_receipt_amt', 'sum'),
    contribution_count=('contb_receipt_amt', 'count')
)

# Sort by total contributions (dollar amount)
top_by_amount = zip_summary.sort_values(by='total_contributions', ascending=False).head(1)

# Sort by number of contributions
top_by_count = zip_summary.sort_values(by='contribution_count', ascending=False).head(1)

# Display both
print("ZIP code with highest total dollar amount:")
print(top_by_amount)

print("\nZIP code with highest count of contributions:")
print(top_by_count)





ZIP code with highest total dollar amount:
           total_contributions  contribution_count
zipcode_5                                         
94110               294,061.13                4046

ZIP code with highest count of contributions:
           total_contributions  contribution_count
zipcode_5                                         
94110               294,061.13                4046


- **3a answer here:** 

**3b.** What day(s) of the month do most people donate? (10 points)

In [ ]:
# 3b YOUR CODE HERE
# Count donations by day of the month
donation_day_counts = bernie['donation_day'].value_counts().sort_values(ascending=False)

# Display top 5 most common donation days
print(" Most common donation days of the month:")
print(donation_day_counts.head(5))



📅 Most common donation days of the month:
donation_day
29    23508
30    22899
31    19486
14    17793
8     16774
Name: count, dtype: int64


- **3b answer here:** 

## If you have feedback for this homework, please submit it using the link below:

http://goo.gl/forms/74yCiQTf6k